In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Thyroid_Cancer/GSE151179'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene and miRNA expression in radioiodine refractory and avid papillary thyroid carcinomas (gene expression dataset)"
!Series_summary	"We performed gene and miRNA expression profiling in a series of 39 papillary thyroid carcinomas (PTCs) and 13 matched non-neoplastic thyroids derived from PTC patients with metastatic disease and submitted to radioiodine (RAI) treatment."
!Series_overall_design	"Gene and miRNA expression profiles were established by microarray analysis in a retrospective series of 52 snap-frozen thyroid samples including 35 tissues collected before RAI treatment (17 primary PTC tumors, 5 synchronous lymph node metastases (LNMs), and 13 matched non-neoplastic thyroids included as control) and 17 RAI-refractory LNMs collected as successive surgery following RAI treatment. Patients were stratified based on RAI uptake at the metastatic site and on RAI response in either avid or refractory, displaying disease remission or persistance, re

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Based on the background information, gene expression data is present

# Column for 'Thyroid_Cancer'
trait_row = 1  # tissue type
trait_unique_values = ['Primary tumor', 'synchronous lymph node metastasis', 'lymph node metastasis post RAI', 'lymph node metastasis_1 post RAI', 'lymph node metastasis_2 post RAI', 'non-neoplastic thyroid']

# Column for 'age'
age_row = None  # Age information is not present in sample characteristics

# Column for 'gender'
gender_row = None  # Gender information is not present in sample characteristics

# Conversion functions

def convert_trait(value):
    value = value.split(': ')[1]  # Extract the actual value
    if "Primary tumor" in value or "metastasis" in value:
        return 1
    elif "non-neoplastic thyroid" in value:
        return 0
    return None

def convert_age(value):
    return None  # Age information not available

def convert_gender(value):
    return None  # Gender information not available

# Save cohort information
save_cohort_info('GSE151179', './preprocessed/Thyroid_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save clinical feature data if available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Thyroid_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Thyroid_Cancer/trait_data/GSE151179.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4567912': [1], 'GSM4567913': [1], 'GSM4567914': [1], 'GSM4567915': [1], 'GSM4567916': [1], 'GSM4567917': [1], 'GSM4567918': [1], 'GSM4567919': [1], 'GSM4567920': [1], 'GSM4567921': [1], 'GSM4567922': [1], 'GSM4567923': [1], 'GSM4567924': [1], 'GSM4567925': [1], 'GSM4567926': [1], 'GSM4567927': [1], 'GSM4567928': [1], 'GSM4567929': [1], 'GSM4567930': [1], 'GSM4567931': [1], 'GSM4567932': [1], 'GSM4567933': [1], 'GSM4567934': [1], 'GSM4567935': [1], 'GSM4567936': [1], 'GSM4567937': [1], 'GSM4567938': [1], 'GSM4567939': [1], 'GSM4567940': [1], 'GSM4567941': [1], 'GSM4567942': [1], 'GSM4567943': [1], 'GSM4567944': [1], 'GSM4567945': [1], 'GSM4567946': [1], 'GSM4567947': [1], 'GSM4567948': [1], 'GSM4567949': [1], 'GSM4567950': [1], 'GSM4567951': [0], 'GSM4567952': [0], 'GSM4567953': [0], 'GSM4567954': [0], 'GSM4567955': [0], 'GSM4567956': [0], 'GSM4567957': [0], 'GSM4567958': [0], 'GSM4567959': [0], 'GSM4567960': [0], 'GSM4567961': [0], 'GSM4567962': [0], 'GSM4567963': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['23064070', '23064071', '23064072', '23064073', '23064074', '23064075',
       '23064076', '23064077', '23064078', '23064079', '23064080', '23064081',
       '23064083', '23064084', '23064085', '23064086', '23064087', '23064088',
       '23064089', '23064090'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member 5 [gene_biotype:protein_coding transcript_biotype:protein_coding] // chr1 // 10

### Step 6: Gene Identifier Mapping

In [7]:
# The identifiers from STEP3 seem to match 'ID' from the annotation, and gene symbols are stored in 'SPOT_ID'

identifier_key = 'ID'
gene_symbol_key = 'SPOT_ID'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Thyroid_Cancer/gene_data/GSE151179.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Thyroid_Cancer')

# 4. Save the cohort information.
save_cohort_info('GSE151179', './preprocessed/Thyroid_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Thyroid_Cancer/GSE151179.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
